In [ ]:
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Restaurant Reservation</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f4f4f4;
        }
        .container {
            max-width: 500px;
            margin: 50px auto;
            padding: 20px;
            background-color: #fff;
            border-radius: 8px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        h2 {
            text-align: center;
        }
        form {
            margin-top: 20px;
        }
        label {
            display: block;
            margin-bottom: 8px;
        }
        input[type="text"],
        input[type="time"] {
            width: 100%;
            padding: 10px;
            margin-bottom: 15px;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        input[type="submit"] {
            width: 100%;
            padding: 10px;
            background-color: #4caf50;
            color: #fff;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }
        input[type="submit"]:hover {
            background-color: #45a049;
        }
    </style>
</head>
<body>
    <div class="container">
        <h2>Restaurant Reservation</h2>
        <form id="reservationForm">
            <label for="name">Name:</label>
            <input type="text" id="name" name="name" required>

            <label for="time">Reservation Time:</label>
            <input type="time" id="time" name="time" required>

            <input type="submit" value="Make Reservation">
        </form>
    </div>

    <script>
        // Function to handle form submission
        document.getElementById("reservationForm").addEventListener("submit", function(event) {
            event.preventDefault(); // Prevent the form from submitting normally
            
            // Get form data
            var name = document.getElementById("name").value;
            var time = document.getElementById("time").value;

            // Store data in local storage
            var reservation = {
                name: name,
                time: time
            };
            localStorage.setItem("reservation", JSON.stringify(reservation));

            // Optional: You can redirect the user to another page or perform any other action here

            // For demonstration purposes, just alert the user
            alert("Reservation made successfully!");
        });
    </script>
</body>
</html>

In [ ]:
const express = require('express');
const bodyParser = require('body-parser');
const mongoose = require('mongoose');

const app = express();
const PORT = process.env.PORT || 3000;

// Connect to MongoDB
mongoose.connect('mongodb://localhost:27017/restaurant_reservation', { useNewUrlParser: true, useUnifiedTopology: true });
const db = mongoose.connection;

db.on('error', console.error.bind(console, 'MongoDB connection error:'));

// Create Reservation schema and model
const reservationSchema = new mongoose.Schema({
    name: String,
    time: String
});

const Reservation = mongoose.model('Reservation', reservationSchema);

// Middleware
app.use(bodyParser.urlencoded({ extended: false }));
app.use(bodyParser.json());

// API endpoints
app.post('/api/reservations', (req, res) => {
    const { name, time } = req.body;
    const newReservation = new Reservation({ name, time });
    newReservation.save((err, savedReservation) => {
        if (err) {
            console.error(err);
            res.status(500).json({ error: 'Failed to save reservation' });
        } else {
            res.status(201).json(savedReservation);
        }
    });
});

// Start server
app.listen(PORT, () => {
    console.log(`Server is running on port ${PORT}`);
});


This is backend made by howie for the idea of a table where you can sign in time and name for any resturaunt you want to eat at. IF previous person has time and you reserve before that, will error out.

In [ ]:
const express = require('express');
const bodyParser = require('body-parser');
const mongoose = require('mongoose');

const app = express();
const PORT = process.env.PORT || 3000;

// Connect to MongoDB
mongoose.connect('mongodb://localhost:27017/restaurant_reservation', { useNewUrlParser: true, useUnifiedTopology: true });
const db = mongoose.connection;

db.on('error', console.error.bind(console, 'MongoDB connection error:'));

// Create Reservation schema and model
const reservationSchema = new mongoose.Schema({
    name: String,
    time: String
});

const Reservation = mongoose.model('Reservation', reservationSchema);

// Middleware
app.use(bodyParser.urlencoded({ extended: false }));
app.use(bodyParser.json());

// API endpoints
// Create a reservation
app.post('/api/reservations', (req, res) => {
    const { name, time } = req.body;
    const newReservation = new Reservation({ name, time });
    newReservation.save((err, savedReservation) => {
        if (err) {
            console.error(err);
            res.status(500).json({ error: 'Failed to save reservation' });
        } else {
            res.status(201).json(savedReservation);
        }
    });
});

// Retrieve all reservations
app.get('/api/reservations', (req, res) => {
    Reservation.find({}, (err, reservations) => {
        if (err) {
            console.error(err);
            res.status(500).json({ error: 'Failed to retrieve reservations' });
        } else {
            res.json(reservations);
        }
    });
});

// Retrieve a single reservation by ID
app.get('/api/reservations/:id', (req, res) => {
    const { id } = req.params;
    Reservation.findById(id, (err, reservation) => {
        if (err) {
            console.error(err);
            res.status(500).json({ error: 'Failed to retrieve reservation' });
        } else if (!reservation) {
            res.status(404).json({ error: 'Reservation not found' });
        } else {
            res.json(reservation);
        }
    });
});

// Update a reservation by ID
app.put('/api/reservations/:id', (req, res) => {
    const { id } = req.params;
    const { name, time } = req.body;
    Reservation.findByIdAndUpdate(id, { name, time }, { new: true }, (err, updatedReservation) => {
        if (err) {
            console.error(err);
            res.status(500).json({ error: 'Failed to update reservation' });
        } else if (!updatedReservation) {
            res.status(404).json({ error: 'Reservation not found' });
        } else {
            res.json(updatedReservation);
        }
    });
});

// Delete a reservation by ID
app.delete('/api/reservations/:id', (req, res) => {
    const { id } = req.params;
    Reservation.findByIdAndDelete(id, (err, deletedReservation) => {
        if (err) {
            console.error(err);
            res.status(500).json({ error: 'Failed to delete reservation' });
        } else if (!deletedReservation) {
            res.status(404).json({ error: 'Reservation not found' });
        } else {
            res.json({ message: 'Reservation deleted successfully' });
        }
    });
});

// Start server
app.listen(PORT, () => {
    console.log(`Server is running on port ${PORT}`);
});
